# Setup

set working directory to guitarist-net and add to path

In [1]:
%cd /content/drive/Othercomputers/My MacBook Pro/guitarist-net

/content/drive/Othercomputers/My MacBook Pro/guitarist-net


install requirements

In [2]:
!pip install -r requirements.txt &> /dev/null

imports

In [3]:
import numpy as np
import torch
import matplotlib.pyplot as plt
from plot_listen.listen import play
from plot_listen.plot import plot
from globals import *

from data.hex_string_dataset import HexStringDataset
from ddsp.ddsp_mono_synth import DDSPMonoSynth
from save_load import load_weights_from_file

# Load Hex Dataset

In [4]:
hex_data = HexStringDataset()

Type a number to select an option:
	0)	test_player-00_e-string_1.0s_crepe.npz
	1)	train_player-not00_e-string_1.0s_crepe.npz
	2)	test_player-00_B-string_1.0s_crepe.npz
	3)	train_player-not00_B-string_1.0s_crepe.npz
	4)	test_player-00_G-string_1.0s_crepe.npz
	5)	train_player-not00_G-string_1.0s_crepe.npz
	6)	test_player-00_D-string_1.0s_crepe.npz
	7)	train_player-not00_D-string_1.0s_crepe.npz
	8)	test_player-00_A-string_1.0s_crepe.npz
	9)	train_player-not00_A-string_1.0s_crepe.npz
	10)	test_player-00_E-string_1.0s_crepe.npz
	11)	train_player-not00_E-string_1.0s_crepe.npz
	12)	test_player-00_hex_1.0s.npz
	13)	train_player-not00_hex_1.0s.npz
Selection: 12
test_player-00_hex_1.0s.npz


In [5]:
data_item = hex_data[0]

## Inspect Data Item

In [6]:
for key in data_item.keys():
    print(key, data_item[key].shape)

mix_audio (16000,)
mic_audio (16000,)
mix_loudness (250,)
mic_loudness (250,)
E_audio (16000,)
E_f0 (250,)
E_voicing (250,)
E_loudness (250,)
A_audio (16000,)
A_f0 (250,)
A_voicing (250,)
A_loudness (250,)
D_audio (16000,)
D_f0 (250,)
D_voicing (250,)
D_loudness (250,)
G_audio (16000,)
G_f0 (250,)
G_voicing (250,)
G_loudness (250,)
B_audio (16000,)
B_f0 (250,)
B_voicing (250,)
B_loudness (250,)
e_audio (16000,)
e_f0 (250,)
e_voicing (250,)
e_loudness (250,)


In [7]:
play(data_item["mix_audio"])

In [8]:
play(data_item["mic_audio"])

In [9]:
play(data_item["E_audio"])

In [10]:
play(data_item["A_audio"])

In [11]:
play(data_item["D_audio"])

In [12]:
play(data_item["G_audio"])

In [13]:
play(data_item["B_audio"])

In [14]:
play(data_item["e_audio"])

# Load Trained String Synths

In [6]:
decoders = {}

for string_let in GUITAR_STRING_LETTERS:
    string_model = DDSPMonoSynth()
    string_model.load_state_dict(load_weights_from_file())
    string_model.eval()
    decoders[string_let] = string_model

Type a number to select an option:
	0)	2022-07-08 08:08PM DDSP Mono Synth : training test
	1)	2022-07-09 01:40PM DDSP Mono Synth : load model and continue training
	2)	2022-07-09 11:45PM DDSP Mono Synth : resume training (third time)
	3)	2022-07-10 10:44AM DDSP Mono Synth : train (4th time)
	4)	2022-07-19 04:05PM DDSP Mono Synth Improved : synth_improvement
	5)	2022-07-19 06:56PM DDSP Mono Synth : train with loudness centered and 32 osc + 32 filter coeffs
	6)	2022-07-20 10:48AM DDSP Mono Synth : train low E
	7)	2022-07-20 12:24PM DDSP Mono Synth Improved : improve 32 param synth - e string
	8)	2022-07-20 07:03PM DDSP Mono Synth : train with lr scheduler
	9)	2022-07-22 11:34AM DDSP Mono Synth : train low E for a while
	10)	2022-07-25 06:42PM DDSP Mono Synth : train with Leaky ReLU
	11)	2022-07-26 04:41PM DDSP Mono Synth : try training with test dataset (one guitarist)
	12)	2022-07-27 06:09PM DDSP Mono Synth : folder deleting, harmonic + noise saving
	13)	2022-07-27 06:12PM DDSP Mono Syn

# Feed Data Into Synths

In [74]:
data_item = hex_data[320]
play(data_item['mix_audio'])

In [75]:
output_audios = []
for string_let in GUITAR_STRING_LETTERS:
    f0 = torch.tensor(data_item[f"{string_let}_f0"]).unsqueeze(0).to(DEVICE)  # cast as tensor, and add batch dim of 1, send to GPU
    loudness = torch.tensor(data_item[f"{string_let}_loudness"]).unsqueeze(0).to(DEVICE) # cast as tensor, and add batch dim of 1, send to GPU
    decoders[string_let].to(DEVICE)
    output_audio = decoders[string_let](f0,loudness)
    output_audios.append(output_audio["reverb_audio"][0]) # get the reverb audio, and remove the batch idx

summed_audio = torch.sum(torch.stack(output_audios), dim=0)
play(summed_audio)